<a href="https://colab.research.google.com/github/yunju-1118/EWHA/blob/main/seq_classificaiton_mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **MNIST data**
**RNN** 이용

first row => first time step

second row => second time step

.
.
.

28th row => 28th time step

In [1]:
import torch
import torch.nn as nn
import torch.functional as F

import torchvision.transforms as transforms
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader

In [2]:
import numpy as np

path = './datasets/'

transform = transforms.Compose([transforms.ToTensor()])

train_data = MNIST(root=path, train=True, transform=transform, download=True)
test_data = MNIST(root=path, train=False, transform=transform, download=True)

batch_size = 100

train_loader = DataLoader(dataset=train_data, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_data, batch_size=batch_size, shuffle=False)

_, seq_len, input_size = train_data[0][0].shape
output_shape = len(train_data.classes)

100%|██████████| 9.91M/9.91M [00:00<00:00, 17.9MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 484kB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 4.42MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 9.75MB/s]


In [3]:
if torch.backends.mps.is_available():
    device = torch.device('mps:0')
elif torch.cuda.is_available():
    device = torch.device('cuda:0')
else: device=torch.device('cpu')

print(device)

cuda:0


hidden state의 input dimension을 정의해주어야 함

In [4]:
_,seq_len, input_size = train_data[0][0].shape # 1, 28, 28
output_shape = len(train_data.classes)

hidden_size = input_size*2

model_name = "rnn"

### **RNNcell 이용**

In [8]:
class RNNClassifier(nn.Module):
    def __init__(self, input_size, hidden_size):
        super().__init__()

        self.input_size = input_size
        self.hidden_size = hidden_size

        self.cell = nn.RNNCell(input_size=self.input_size,
                               hidden_size=self.hidden_size)

        self.fc = nn.Linear(self.hidden_size, output_shape)

        def forward(self,x):
            # x.shape = (100,1,28,28) (변형)=> x.squeeze() -> 28, 100, 28 = seq_len, batch_size, input_dim
            x = x.reshape(-1, seq_len, self.input_size).permute((1,0,2)) # 28,100,28
            hidden_state = torch.zeros(batch_size, self.hidden_size).to(device)
            for i in range(seq_len):
                hidden_state = self.cell(x[i], hidden_state)
            out = self.fc(hidden_state)

            return out

**RNN** => 쉽게 말해 '수열' 같음

맨 처음에 존재하는 (initial) hidden state가 존재해야 남은 hidden state를 update할 수 있음

=> zero vector 혹은 random한 vector로 가정

lstm은 cell state만 추가

In [10]:
class LSTMClassifier(nn.Module):
    def __init__(self, input_size, hidden_size):
        super().__init__()

        self.input_size = input_size
        self.hidden_size = hidden_size

        self.cell = nn.LSTMCell(input_size = self.input_size,
                                hidden_size = self.hidden_size)
        self.fc = nn.Linear(self.hidden_size, output_shape)

        def forward(self, x):
            x = x.reshape(-1, seq_len, self.input_size).permute((1,0,2))
            hidden_state = torch,zeros(batch_size, self.hidden_size).to(device)
            cell_state = torch,zeros(batch_size, self.hidden_size).to(device)
            for i in range(seq_len):
                hidden_state, cell_state = self.call(x[i],(hidden_state, cell_state))
            out = self.fc(hidden_state)

            return out

In [12]:
class GRUClassifier(nn.Module):
    def __init__(self, input_size, hidden_size):
        super().__init__()

        self.input_size = input_size
        self.hidden_size = hidden_size

        self.cell = nn.GRUCell(input_size=self.input_size,
                               hidden_size=self.hidden_size)

        self.fc = nn.Linear(self.hidden_size, output_shape)

        def forward(self,x):
            x = x.reshape(-1, seq_len, self.input_size).permute((1,0,2)) # 28,100,28
            hidden_state = torch.zeros(batch_size, self.hidden_size).to(device)
            for i in range(seq_len):
                hidden_state = self.cell(x[i], hidden_state)
            out = self.fc(hidden_state)

            return out

cell state 라는 게 따로 존재하지 않아 rnn과 같음

In [13]:
if model_name == "rnn":
    classifier = RNNClassifier
elif model_name == "lstm":
    classifier = LSTMClassifier
elif model_name == "gru":
    classifier = GRUClassifier

## **CIFAR10**

### **RNN 이용**

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import torchvision.transforms as transforms
from torchvision.datasets import CIFAR10
from torch.utils.data import DataLoader

import numpy as np

path = './datasets/'

transform = transforms.Compose([transforms.ToTensor()])

train_data = CIFAR10(root=path, train=True, transform=transform, download=True)
test_data = CIFAR10(root=path, train=False, transform=transform, download=True)

batch_size = 100

train_loader = DataLoader(dataset=train_data, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset = test_data, batch_size=batch_size, shuffle=False)

num_channel, seq_len, input_size = train_data[0][0].shape  # 3*32*32
output_shape = len(train_data.classes)

hidden_size = input_size*2
num_layers=3
batch_first = True
bidirectional = True

model_name = "gru"

In [ ]:
if model_name == "rnn":
    classifier = RNNClassifier
elif model_name == "lstm":
    classifier = LSTMClassifier
elif model_name == "gru":
    classifier = GRUClassifier

In [ ]:
class RNNClassifier(nn.Module):
    def __init__(self.num_channel, input_size, hidden_size, num_layers=1,
                 batch_first=True, bidirectional=False):
        super().__init__()

        self.num_channel = num_channel
        self.input_size = input_size
        self.hidden_size hidden_size
        self.num_layers = num_layers
        self.batch_first = batch_first
        self.bidirectional bidirectional

        if self.bidirectional:
            self.arrent = 2
            else: by_direction=1

        self.seq = nn.RNN(input_size = input_size*self.num_channel1,
                          hidden_size = self.hidden_size,
                          num_layers = self.num_layers,
                          batch_first = self.both(bidirectional))

        self.fc = nn.Linear(self.hidden_size*self.direction,output_shape)

    def forward(self, x):
        # x.shape(100, 3, 32, 32) = batch_size, num_channel, seq_len, input_dim)
        x = x.permute((0,2,3,1)).reshape(-1, seq_len,False, _direction,out_put shape)
        h0 = torch.zeros(self.direction*self.num_layers, batch_size, self.hidden_size)
        out, hidden = self.seq(x,h0. detach().to(device))
        out = out[:,-1,:].squeeze()
        out = self.fc(out)

        return out

bidirenctional = True 시,

오류 발생 => 어떤 코드 수정?

In [ ]:
class LSTMClassifier(nn.Module):
    def __init__(self.num_channel, input_size, hidden_size, num_layers=1,
                 batch_first=True, bidirectional=False):
        super().__init__()

        self.num_channel = num_channel
        self.input_size = input_size
        self.hidden_size hidden_size
        self.num_layers = num_layers
        self.batch_first = batch_first
        self.bidirectional bidirectional

        if self.bidirectional:
            self.arrent = 2
            else: by_direction=1

        self.seq = nn.LSTM(input_size = input_size*self.num_channel1,
                          hidden_size = self.hidden_size,
                          num_layers = self.num_layers,
                          batch_first = self.both(bidirectional))

        self.fc = nn.Linear(self.hidden_size*self.direction,output_shape)

    def forward(self, x):
        # x.shape(100, 3, 32, 32) = batch_size, num_channel, seq_len, input_dim)
        x = x.permute((0,2,3,1)).reshape(-1, seq_len,False, _direction,out_put shape)
        h0 = torch.zeros(self.direction*self.num_layers, batch_size, self.hidden_size)
        c0 = torch.zeros(self.direction*self,num_layers, batch_size, self.hidden_size)
        out, hidden = self.seq(x,h0. detach().to(device))
        out = out[:,-1,:].squeeze()
        out = self.fc(out)

        return out